## Demo Script Localization

Given a demo script document, generate its localized version. The localized version is expected to satisfy following objectives:
- Translate the language of the demo script to the language specified by the user.
- Use names from the country specified by the user in the demo script.

In [1]:
import streamlit as st
import openai
import os
from docx import Document
import concurrent.futures
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


def localize_demo_script(input_path, target_language, target_country, target_name):
    doc = Document(input_path)
    paragraphs = [para.text for para in doc.paragraphs]

    batch_size = 5  # Number of paragraphs to process in each batch
    batched_paragraphs = [paragraphs[i:i+batch_size] for i in range(0, len(paragraphs), batch_size)]

    translated_paragraphs = []
    for batch in batched_paragraphs:
        translations = generate_batch_translation(batch, target_language)
        translated_paragraphs.extend(translations)

    for i in range(len(translated_paragraphs)):
        translated_paragraphs[i] = translated_paragraphs[i].replace(target_name, f"{{Name from {target_country}}}")

    localized_path = f"localized_script_{target_language}_{target_country}.docx"
    save_as_docx(translated_paragraphs, localized_path)

    return localized_path


In [2]:
def generate_batch_translation(batch, target_language):
    batch_text = "\n".join(batch)
    translation = openai.Completion.create(
        engine="text-similarity-davinci-001",
        prompt=f"Translate the following text to {target_language}:\n{batch_text}",
        max_tokens=150 * len(batch),  # Adjust max_tokens for batch size
        stop=None
    )
    return [choice.text.strip() for choice in translation.choices]

In [3]:
def save_as_docx(content, output_path):
    doc = Document()
    for para in content:
        doc.add_paragraph(para)
    doc.save(output_path)

In [5]:
def main():
    st.title("Demo Script Localization")

    uploaded_file = st.file_uploader("Upload the demo script (DOCX file)", type=["docx"])
    target_language = st.text_input("Target Language (e.g., Spanish)")
    target_country = st.text_input("Target Country (e.g., India)")
    target_name = st.text_input(f"Name from {target_country} (e.g., Rahul Sharma)")

    if st.button("Localize"):
        if uploaded_file and target_language and target_country and target_name:
            localized_path = localize_demo_script(uploaded_file, target_language, target_country, target_name)
            st.success(f"Localized script saved as: {localized_path}")
            st.download_button("Download Localized Script", localized_path)

if __name__ == "__main__":
    main()